In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
# os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")

In [3]:
from langchain_community.document_loaders import WebBaseLoader


In [4]:
loader=WebBaseLoader("https://medium.com/@tejpal.abhyuday/retrieval-augmented-generation-rag-from-basics-to-advanced-a2b068fd576c")

In [5]:
loader

In [6]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://medium.com/@tejpal.abhyuday/retrieval-augmented-generation-rag-from-basics-to-advanced-a2b068fd576c', 'title': 'Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | Medium', 'description': 'In the context of Retrieval-Augmented Generation (RAG) in Large Language Models (LLMs), here are the advantages and disadvantages of the three approaches:', 'language': 'en'}, page_content='Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | MediumOpen in appSign upSign inWriteSign upSign inRetrieval-Augmented Generation (RAG) from basics to advancedTejpal Kumawat·Follow12 min read·Feb 14, 2024--3ListenShareIntroduction:Retrieval-Augmented Generation (RAG) is a technique that enhances language model generation by incorporating external knowledge.This is typically done by retrieving relevant information from a large corpus of documents and using that information to inform the generation process.Challen

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [8]:
documents

[Document(metadata={'source': 'https://medium.com/@tejpal.abhyuday/retrieval-augmented-generation-rag-from-basics-to-advanced-a2b068fd576c', 'title': 'Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | Medium', 'description': 'In the context of Retrieval-Augmented Generation (RAG) in Large Language Models (LLMs), here are the advantages and disadvantages of the three approaches:', 'language': 'en'}, page_content='Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | MediumOpen in appSign upSign inWriteSign upSign inRetrieval-Augmented Generation (RAG) from basics to advancedTejpal Kumawat·Follow12 min read·Feb 14, 2024--3ListenShareIntroduction:Retrieval-Augmented Generation (RAG) is a technique that enhances language model generation by incorporating external knowledge.This is typically done by retrieving relevant information from a large corpus of documents and using that information to inform the generation process.Challen

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [11]:
vectorstoredb

In [22]:
query=" GPT4-Turbo Introduction"

In [23]:
result=vectorstoredb.similarity_search(query)

In [24]:
result

[Document(metadata={'source': 'https://medium.com/@tejpal.abhyuday/retrieval-augmented-generation-rag-from-basics-to-advanced-a2b068fd576c', 'title': 'Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | Medium', 'description': 'In the context of Retrieval-Augmented Generation (RAG) in Large Language Models (LLMs), here are the advantages and disadvantages of the three approaches:', 'language': 'en'}, page_content='Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | MediumOpen in appSign upSign inWriteSign upSign inRetrieval-Augmented Generation (RAG) from basics to advancedTejpal Kumawat·Follow12 min read·Feb 14, 2024--3ListenShareIntroduction:Retrieval-Augmented Generation (RAG) is a technique that enhances language model generation by incorporating external knowledge.This is typically done by retrieving relevant information from a large corpus of documents and using that information to inform the generation process.Challen

In [25]:
result[0].page_content

'Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | MediumOpen in appSign upSign inWriteSign upSign inRetrieval-Augmented Generation (RAG) from basics to advancedTejpal Kumawat·Follow12 min read·Feb 14, 2024--3ListenShareIntroduction:Retrieval-Augmented Generation (RAG) is a technique that enhances language model generation by incorporating external knowledge.This is typically done by retrieving relevant information from a large corpus of documents and using that information to inform the generation process.Challenge:Clients often have vast proprietary documents.Extracting specific information is like finding a needle in a haystack.2. GPT4-Turbo Introduction:OpenAI’s GPT4-Turbo can process large documents.3. Efficiency Issue:“Lost In The Middle” phenomenon hampers efficiency.Model forgets content in the middle of its contextual window.4. Alternative Approach — Retrieval-Augmented-Generation (RAG):Create an index for each document paragraph.Swiftly'

In [26]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x0000023D648DE5C0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000023D648DF940> root_client=<openai.OpenAI object at 0x0000023D648DF2E0> root_async_client=<openai.AsyncOpenAI object at 0x0000023D648DF8B0> model_name='gpt-4' model_kwargs={} openai_api_key=SecretStr('**********')


In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
   Answer the following question based only on the provided context:
   <context>
   {context}
   </context>
    """
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n   Answer the following question based only on the provided context:\n   <context>\n   {context}\n   </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000023D648DE5C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000023D648DF940>, root_client=<openai.OpenAI object at 0x0000023D648DF2E0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023D648DF8B0>, model_name='gpt-4', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name'

In [28]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"GPT4-Turbo Introduction",
    "context":[Document(page_content="GPT4-Turbo Introduction Typically, queries are embedded using models like BERT for generating dense vector embeddings.")]
})

'The context implies that queries are typically embedded using models like BERT to generate dense vector embeddings in the introduction of GPT4-Turbo.'

In [29]:
vectorstoredb

In [30]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [31]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023D27FD7EE0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n   Answer the following question based only on the provided context:\n   <context>\n   {context}\n   </context>\n    '), additional_kwargs={})])
  

In [32]:
response=retrieval_chain.invoke({"input":"GPT4-Turbo Introduction"})
response['answer']

'The Retrieval-Augmented Generation (RAG) is a technique designed to enhance language model generation by incorporating external knowledge. It works by retrieving relevant information from a large corpus of documents and using that information to inform the generation process. The approach includes creating an index for each document paragraph, swiftly identifying pertinent paragraphs, and feeding selected paragraphs into a Large Language Model (LLM) like GPT4. This process prevents information overload and enhances result quality by providing only relevant paragraphs. However, there are challenges, such as the "Lost In The Middle" phenomenon, where the model forgets content in the middle of its contextual window, hampering efficiency. Researchers found that RAG performs best when the relevant information is at the beginning of the context, and performance decreases with an increase in context length.'

In [33]:
response

{'input': 'GPT4-Turbo Introduction',
 'context': [Document(metadata={'source': 'https://medium.com/@tejpal.abhyuday/retrieval-augmented-generation-rag-from-basics-to-advanced-a2b068fd576c', 'title': 'Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | Medium', 'description': 'In the context of Retrieval-Augmented Generation (RAG) in Large Language Models (LLMs), here are the advantages and disadvantages of the three approaches:', 'language': 'en'}, page_content='Retrieval-Augmented Generation (RAG) from basics to advanced | by Tejpal Kumawat | MediumOpen in appSign upSign inWriteSign upSign inRetrieval-Augmented Generation (RAG) from basics to advancedTejpal Kumawat·Follow12 min read·Feb 14, 2024--3ListenShareIntroduction:Retrieval-Augmented Generation (RAG) is a technique that enhances language model generation by incorporating external knowledge.This is typically done by retrieving relevant information from a large corpus of documents and using that inf